# Understanding Trends in Company Valuation with NLP - Part 1: NLP Setup for Company Earnings Analysis

We will use [SageMaker Studio Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines.html) to automate and run jobs in our pipleline. SageMaker Pipelines uses purpose-built docker containers behind the scene to run jobs (aka [Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html)) in a sequence that you define (much like a DevOps CI/CD pipeline). You can build our own docker container with Python3, [Boto3 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) and [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) installs, so that you can make use of them to make calls to Amazon Fraud Detector APIs via the AWS Boto3 library and access SageMaker constructs, such as Feature Store via custom data processing scripts.

## Create a custom container

To achieve that, you first have to build a docker image and push it to an [ECR (Elastic Container Registry)](https://aws.amazon.com/ecr/) repo in your account. Typically, this can be done using the `docker` CLI and `aws cli` in your local machine pretty easily. However, SageMaker makes it even easier to use this in the studio environment to build, create, and push any custom container to your ECR repository using a purpose-built tool known as `sagemaker-studio-image-build`, and use the custom container image in your notebooks for your ML projects. 

For more information on this, see [Using the Amazon SageMaker Studio Image Build CLI to build container images from your Studio notebooks](https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/).

Next, install this required CLI tool into the SageMaker environment.

In [ ]:
import boto3
region = boto3.Session().region_name
MY_ACCOUNT = boto3.client('sts').get_caller_identity().get('Account')

nlp_script_processor = f"nlp-script-processor:1.0"

CONTAINER_IMAGE_URI=f"{MY_ACCOUNT}.dkr.ecr.{region}.amazonaws.com/{nlp_script_processor}"
CONTAINER_IMAGE_URI

In [ ]:
# Install sagemaker-studio-image-build CLI tool
!pip install sagemaker-studio-image-build

---

### 1. Grant appropriate permissions to SageMaker

In order to use `sagemaker-studio-image-build`, we need to first add permissions to SageMaker's IAM role so that it may perform actions on your behalf. Specifically, you would add Amazon ECR and Amazon CodeBuild permissions to it. Add the <font color='green'> AmazonEC2ContainerRegistryFullAccess </font> and <font color='green'> AWSCodeBuildAdminAccess </font> policies to your SageMaker default role.

<img src="./images/iam-perm.png" width="500" height="600"/>


In addition to this, you will also have to add the `iam:PassRole` permission to the SageMaker Studio execution role. Add the following policy as an inline policy to the SageMaker Studio Execution role using the AWS IAM console.

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::*:role/*",
            "Condition": {
                "StringLikeIfExists": {
                    "iam:PassedToService": "codebuild.amazonaws.com"
                }
            }
        }
    ]
}
```

Also, you must add a trust relationship in the SageMaker Studio Execution role to allow CodeBuild to assume this role. To add a trust relationship, do the following: 

1. Navigate to IAM Console
2. Search for your SageMaker execution role. (You can find your SageMaker execution role name from SageMaker Studio console)
3. Click on the "Trust Relationships" tab > Click the "Edit Trust relationship" button
4. Add the following Trust relationship to any pre-existing trust relationship

    ```json
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "codebuild.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
    ```

In a typical situation, your final trust relationship should look something like the following:

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "sagemaker.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "codebuild.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```

<img src="./images/trust.png" width="500" height="600"/>

<div class="alert alert-warning"> <h4 color="red"><strong>💡 NOTE </strong></h4>
IAM Policies described in this notebook can be overly permissive. Please practice caution in setting up IAM Roles with them. For more information about fine-grained permissions for the sagemaker-studio-image-build tool, see <a href="https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/">Using the Amazon SageMaker Studio Image Build CLI to build container images from your Studio notebooks</a>. For best practices on SageMaker security, IAM roles, and policies, see <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/security_iam_id-based-policy-examples.html#security_iam_service-with-iam-policy-best-practices">Policy Best Practices</a> in the Amazon SageMaker Developer Guide.
</div>

---

### 2. Build a custom Docker image

We now build a custom [Dockerfile](https://docs.docker.com/engine/reference/builder/) and use the CLI tool to build the image from the Dockerfile. Our docker image is going to be pretty simple, it will be a copy of the open source [python:3.7-slim-buster](https://github.com/docker-library/python/blob/117d4e375b86cdbe1853930478d0d07d7d5701f7/3.7/buster/slim/Dockerfile) image and contain an installation of [Boto3 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html), [SageMaker SDK](https://github.com/aws/sagemaker-python-sdk), Pandas, and NumPy.

For our NLP pipeline, we have a number of tasks that depend on Boto3 and SageMaker SDK. We will also use the [SageMaker JumpStart Industry Python SDK](https://sagemaker-jumpstart-industry-pack.readthedocs.io/en/latest/notebooks/finance/notebook1/SEC_Retrieval_Summarizer_Scoring.html) to download 10k/10Q reports from SEC's EDGAR system. We install all of these dependencies in the container, and use the custom container in our [`ScriptProcessor` step](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-container-run-scripts.html) in our pipelines.

In [ ]:
%%writefile Dockerfile
FROM python:3.7-slim-buster

RUN pip3 install smjsindustry==1.0.0 requests botocore boto3>=1.15.0 sagemaker pandas numpy transformers typing sentencepiece nltk
RUN python3 -c "import nltk; nltk.download('punkt')"
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

The code cell above will create a `Dockerfile` in the local project's directory. We can then run the `sm-docker build` command to build, and publish our image. This single command will take care of building the Docker image and publishing it to a [private ECR Repository](https://docs.aws.amazon.com/AmazonECR/latest/userguide/Repositories.html) in your current region (i.e. your SageMaker Studio's default Region). 

<font color='red'> NOTE: </font> You must execute the code cell above to run the following cells. the `sm-docker build` command reads the `Dockerfile` to create the docker image. To ensure that the code above ran successfully, please verify that you have a file named `Dockerfile` is under the same directory where this notebook is located in the left navigation pane of Studio. This project already includes the Dockerfile, however, if you modify the code cell above, it would be a good idea to verify if the contents of the Dockerfile were updated correctly.


In [ ]:
%%time

!sm-docker build . --repository $nlp_script_processor

Running the command in the preceding code cell prints log lines in the notebook ending with three lines like the following example:

```sh
[Container] 2021/05/15 03:19:43 Phase complete: POST_BUILD State: SUCCEEDED
[Container] 2021/05/15 03:19:43 Phase context status code:  Message:
Image URI: <ACCOUNT_ID>.dkr.ecr.<REGION>.amazonaws.com/sagemaker-studio-d-xxxxxxxxx:default-<xxxxxxxxxxx>
```

We will need the `Image URI` for our SageMaker pipeline setup. You can also find this image URI from the [ECR Console](https://console.aws.amazon.com/ecr/repositories) (make sure the correct region is selected in the ECR console).

Initialize a variable with the Docker image URI that we built in the previous step, we will also store this variable in cache.

In [ ]:
# Copy the Image URI from the log output below
#------------------------------------------------

# CONTAINER_IMAGE_URI= "<ACCOUNT_ID>.dkr.ecr.<REGION>.amazonaws.com/sagemaker-studio-d-xxxxxxxxx:default-<xxxxxxxxxxx>"

%store CONTAINER_IMAGE_URI
CONTAINER_IMAGE_URI

---

## Conclusion

In this notebook, we did the following:

* Installed the `sagemaker-studio-image-build` CLI tool that helps us build and publish custom docker images for our ML Workstream.
* Set up IAM permissions for the CLI tool.
* Built the docker image using the `sm-docker build` command that includes Boto3 and SageMaker SDK libraries.
* Initialized the variable `CONTAINER_IMAGE_URI` with the resulting image URI and stored it in cache (notebook magic command) for use later.